In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import html5lib

print('Libraries imported.')

Libraries imported.


## Scrapping the Web page: 

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(url)
soup = BeautifulSoup(results.content, 'html.parser')
# print(soup.prettify())

In [4]:
table_html = soup.find('table')
table_pd = pd.read_html(str(table_html), flavor = 'html5lib')[0]
table_pd.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Preparing the Dataframe:

In [5]:
table_pd = table_pd[table_pd.Borough!='Not assigned'].reset_index(drop=True)
table_pd = table_pd.sort_values('Postal Code',axis = 0, ascending = True, ignore_index = True)
table_pd.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
table_pd.shape

(103, 3)

In [6]:
postal_code_pd = pd.read_csv('Geospatial_Coordinates.csv')

In [7]:
table_pd = table_pd.join(postal_code_pd[['Latitude','Longitude']])

In [8]:
table_pd.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Plotting map of Toronto with labels:

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

In [34]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lon, code, borough in zip(table_pd['Latitude'],table_pd['Longitude'], table_pd['Postal Code'], table_pd['Borough']):
    # print('{}, {}, {}'.format(label, lon, lat))
    label = '{}, {}'.format(code, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

## Exploring venues for each postal code:

### Setting up the Four Square API:

In [10]:
CLIENT_ID = 'TWQNBUQDQIII1KZJ0RTLFNG0NR15AVJIZU5KFPHM3ANEHMHU' # your Foursquare ID
CLIENT_SECRET = 'VD0GO1D1LCR1T5GYDJHBVXS4REOQD4KVWSADZAZGKXOAQ1DU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Getting the top 100 venues that are in Postale Code M5N within a radius of 500 meters (Test).

In [29]:
LIMIT = 100
radius = 500

neighborhood_latitude = list(table_pd.loc[table_pd['Postal Code'] == 'M5N']['Latitude'])[0]
neighborhood_longitude = list(table_pd.loc[table_pd['Postal Code'] == 'M5N']['Longitude'])[0]

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)


In [30]:
results = requests.get(url).json()

In [31]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-5ae8cfe98496ca002c65d83a-0,0,"[{'summary': 'This spot is popular', 'type': '...",5ae8cfe98496ca002c65d83a,Ceiling Champions,505 The West Mall Suite #207,43.713891,-79.420702,"[{'label': 'display', 'lat': 43.71389073142895...",389,M9C 1G2,CA,Etobicoke,ON,Canada,"[505 The West Mall Suite #207, Etobicoke ON M9...","[{'id': '545419b1498ea6ccd0202f58', 'name': 'H...",0,[],NaN
1,e-0-4e6e176c45dd293273b74e3c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4e6e176c45dd293273b74e3c,Rosalind's Garden Oasis,NaN,43.712189,-79.411978,"[{'label': 'display', 'lat': 43.71218888050602...",402,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",0,[],NaN
2,e-0-5733ad6d498ed068d25eef81-2,0,"[{'summary': 'This spot is popular', 'type': '...",5733ad6d498ed068d25eef81,Aquatics Academy Inc.,391 St Clements Ave,43.709951,-79.412127,"[{'label': 'display', 'lat': 43.70995116372792...",432,M5N 1M2,CA,Toronto,ON,Canada,"[391 St Clements Ave, Toronto ON M5N 1M2, Canada]","[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",0,[],188349696


In [32]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[filtered_columns]

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

In [35]:
nearby_venues.columns = map(str.title,[col_name.split('.')[-1] for col_name in nearby_venues.columns])

In [36]:
nearby_venues.head()

,Name,Categories,Lat,Lng
0,Ceiling Champions,Home Service,43.713891,-79.420702
1,Rosalind's Garden Oasis,Garden,43.712189,-79.411978
2,Aquatics Academy Inc.,Pool,43.709951,-79.412127


In [37]:
# Getting the 100 top venues for each postal code and building a dataframe that groups all the results.
# The resulting dataframe will contain the borough name, Postal code, latitude and longitude of the postal code.
# The venues for each postal code will be added is separate rows. Their categories will be specified as well as their latitude and longitude.

borough_venue = pd.DataFrame()
for postal_code, borough, borough_lat, borough_lng in zip(table_pd['Postal Code'], table_pd['Borough'], table_pd['Latitude'], table_pd['Longitude']):
    print('{},{}'.format(postal_code,borough))
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, borough_lat, borough_lng, VERSION, radius, LIMIT)
    nearby_venues = json_normalize(requests.get(url).json()['response']['groups'][0]['items'])
    # print(len(nearby_venues.columns))
    if len(nearby_venues.columns)>=1:
        # print('yes')
        nearby_venues = nearby_venues[filtered_columns]
        # print(nearby_venues[filtered_columns])
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        nearby_venues.columns = map(str.title,[col_name.split('.')[-1] for col_name in nearby_venues.columns])
        nearby_venues['Postal Code'] = postal_code 
        nearby_venues['Borough'] = borough
        nearby_venues['Postal Code latitude'] = borough_lat 
        nearby_venues['Postal Code longitude'] = borough_lng
        nearby_venues
        # time.sleep(8) 
        borough_venue = borough_venue.append(nearby_venues)

M1B,Scarborough
M1C,Scarborough
M1E,Scarborough
M1G,Scarborough
M1H,Scarborough
M1J,Scarborough
M1K,Scarborough
M1L,Scarborough
M1M,Scarborough
M1N,Scarborough
M1P,Scarborough
M1R,Scarborough
M1S,Scarborough
M1T,Scarborough
M1V,Scarborough
M1W,Scarborough
M1X,Scarborough
M2H,North York
M2J,North York
M2K,North York
M2L,North York
M2M,North York
M2N,North York
M2P,North York
M2R,North York
M3A,North York
M3B,North York
M3C,North York
M3H,North York
M3J,North York
M3K,North York
M3L,North York
M3M,North York
M3N,North York
M4A,North York
M4B,East York
M4C,East York
M4E,East Toronto
M4G,East York
M4H,East York
M4J,East York
M4K,East Toronto
M4L,East Toronto
M4M,East Toronto
M4N,Central Toronto
M4P,Central Toronto
M4R,Central Toronto
M4S,Central Toronto
M4T,Central Toronto
M4V,Central Toronto
M4W,Downtown Toronto
M4X,Downtown Toronto
M4Y,Downtown Toronto
M5A,Downtown Toronto
M5B,Downtown Toronto
M5C,Downtown Toronto
M5E,Downtown Toronto
M5G,Downtown Toronto
M5H,Downtown Toronto
M5J,Downtow

In [38]:
 # Organizing the columns of the resulting dataframe:
 borough_venue = borough_venue[['Borough','Postal Code','Postal Code latitude', 'Postal Code longitude','Name', 'Categories', 'Lat','Lng']].reset_index(drop=True)
 borough_venue.head()

,Borough,Postal Code,Postal Code latitude,Postal Code longitude,Name,Categories,Lat,Lng
0,Scarborough,M1B,43.806686,-79.194353,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Scarborough,M1B,43.806686,-79.194353,Interprovincial Group,Print Shop,43.805630,-79.200378
2,Scarborough,M1C,43.784535,-79.160497,Great Shine Window Cleaning,Home Service,43.783145,-79.157431
3,Scarborough,M1C,43.784535,-79.160497,Royal Canadian Legion,Bar,43.782533,-79.163085
4,Scarborough,M1E,43.763573,-79.188711,RBC Royal Bank,Bank,43.766790,-79.191151


In [39]:
print(borough_venue.shape)
borough_venue['Borough'].unique()

(2139, 8)


array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [40]:
borough_venue.groupby(borough_venue['Borough']).count()

,Postal Code,Postal Code latitude,Postal Code longitude,Name,Categories,Lat,Lng
Borough,,,,,,,
Central Toronto,106,106,106,106,106,106,106
Downtown Toronto,1238,1238,1238,1238,1238,1238,1238
East Toronto,120,120,120,120,120,120,120
East York,71,71,71,71,71,71,71
Etobicoke,77,77,77,77,77,77,77
Mississauga,13,13,13,13,13,13,13
North York,240,240,240,240,240,240,240
Scarborough,92,92,92,92,92,92,92
West Toronto,165,165,165,165,165,165,165


## One hot encoding of the borough and venues dataframe by category venue:

In [41]:
# One hot encoding for all the boroughs:
borough_venue_onehot = pd.get_dummies(borough_venue[['Categories']], prefix="", prefix_sep="")
borough_venue_onehot['Borough'] = borough_venue['Borough']
borough_venue_onehot['Postal Code'] = borough_venue['Postal Code']

# Specifying the order of columns of the resulting dataframe:
column_order = [borough_venue_onehot.columns[-2]] + [borough_venue_onehot.columns[-1]] + list(borough_venue_onehot.columns[:-2])
borough_venue_onehot = borough_venue_onehot[column_order]

borough_venue_onehot.head()

,Borough,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [42]:
# borough_venue_onehot.groupby(borough_venue_onehot['Borough']).count()
borough_venue_onehot['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

### Let's group rows by Postal Code and by taking the mean of the frequency of occurrence of each category:

In [43]:
borough_venue_grouped = borough_venue_onehot.groupby('Postal Code').mean().reset_index()
borough_venue_grouped.head()

,Postal Code,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0

In [76]:
borough_venue_grouped.shape

(100, 272)

### Displaying the most common venues for each postal code:

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['Postal Code'] = borough_venue_grouped['Postal Code']

for ind in np.arange(borough_venue_grouped.shape[0]):
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(borough_venue_grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
1,M1C,Bar,Home Service,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,M1E,Restaurant,Moving Target,Breakfast Spot,Medical Center,Mexican Restaurant,Intersection,Bank,Rental Car Location,Electronics Store,Creperie
3,M1G,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Hakka Restaurant,Gas Station,Bakery,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Drugstore,Donut Shop


## Clustering:

In [46]:
borough_venue_clustering = borough_venue_grouped.drop('Postal Code',1)

In [47]:
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(borough_venue_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 4, 3, 1, 3, 3])

In [191]:
len(kmeans.labels_)

100

In [48]:
# Building a dataframe containing each postal code, its latitude & longitude as well as the cluster label associated to it.
clustered_borough = borough_venue[['Borough','Postal Code','Postal Code latitude', 'Postal Code longitude']].drop_duplicates('Postal Code').reset_index(drop=True)
clustered_borough.insert(0,'Clustering Labels', kmeans.labels_)

In [49]:
clustered_borough = clustered_borough[['Borough','Postal Code','Postal Code latitude', 'Postal Code longitude','Clustering Labels']]
clustered_borough.head()

,Borough,Postal Code,Postal Code latitude,Postal Code longitude,Clustering Labels
0,Scarborough,M1B,43.806686,-79.194353,3
1,Scarborough,M1C,43.784535,-79.160497,3
2,Scarborough,M1E,43.763573,-79.188711,3
3,Scarborough,M1G,43.770992,-79.216917,3
4,Scarborough,M1H,43.773136,-79.239476,3


## Mapping the postal codes once clustered:

In [50]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for borough, lat, lon, poi, cluster in zip(clustered_borough['Borough'], clustered_borough['Postal Code latitude'], clustered_borough['Postal Code longitude'], clustered_borough['Postal Code'], clustered_borough['Clustering Labels']):
    label = '{}, {}'.format(poi, borough)
    label = folium.Popup(label + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters